In [2]:
import folium
import json
import pandas as pd
import numpy as np
import geopandas as gpd
import fiona
import os
import seaborn as sns
import matplotlib.pyplot as plt
from shapely.geometry import Polygon, LineString, Point
from tqdm import tqdm

In [9]:
data = gpd.read_file('data/서울시 격자 인구 데이터/강남서초2030대인구수 - 복사본.geojson')
data

,gid,2030val,layer,geometry
0,다사579454,77.0,"강남구 20,30대 인구수","MULTIPOLYGON (((127.02366 37.50691, 127.02365 ..."
1,다사613448,66.0,"강남구 20,30대 인구수","MULTIPOLYGON (((127.06216 37.50166, 127.06215 ..."
2,다사585452,0.0,"강남구 20,30대 인구수","MULTIPOLYGON (((127.03046 37.50514, 127.03045 ..."
3,다사583456,227.0,"강남구 20,30대 인구수","MULTIPOLYGON (((127.02817 37.50874, 127.02817 ..."
4,다사592432,85.0,"강남구 20,30대 인구수","MULTIPOLYGON (((127.03849 37.48714, 127.03848 ..."
...,...,...,...,...
9128,다사559438,NaN,"서초구 20,30대 인구수","MULTIPOLYGON (((127.00113 37.49240, 127.00112 ..."
9129,다사618387,NaN,"서초구 20,30대 인구수","MULTIPOLYGON (((127.06813 37.44669, 127.06813 ..."
9130,다사559464,NaN,"서초구 20,30대 인구수","MULTIPOLYGON (((127.00097 37.51583, 127.00097 ..."
9131,다사585417,NaN,"서초구 20,30대 인구수","MULTIPOLYGON (((127.03066 37.47359, 127.03065 ..."


In [10]:
data.loc[data['2030val'].apply(pd.isna), '2030val'] = 0
data

,gid,2030val,layer,geometry
0,다사579454,77.0,"강남구 20,30대 인구수","MULTIPOLYGON (((127.02366 37.50691, 127.02365 ..."
1,다사613448,66.0,"강남구 20,30대 인구수","MULTIPOLYGON (((127.06216 37.50166, 127.06215 ..."
2,다사585452,0.0,"강남구 20,30대 인구수","MULTIPOLYGON (((127.03046 37.50514, 127.03045 ..."
3,다사583456,227.0,"강남구 20,30대 인구수","MULTIPOLYGON (((127.02817 37.50874, 127.02817 ..."
4,다사592432,85.0,"강남구 20,30대 인구수","MULTIPOLYGON (((127.03849 37.48714, 127.03848 ..."
...,...,...,...,...
9128,다사559438,0.0,"서초구 20,30대 인구수","MULTIPOLYGON (((127.00113 37.49240, 127.00112 ..."
9129,다사618387,0.0,"서초구 20,30대 인구수","MULTIPOLYGON (((127.06813 37.44669, 127.06813 ..."
9130,다사559464,0.0,"서초구 20,30대 인구수","MULTIPOLYGON (((127.00097 37.51583, 127.00097 ..."
9131,다사585417,0.0,"서초구 20,30대 인구수","MULTIPOLYGON (((127.03066 37.47359, 127.03065 ..."


In [11]:
# 격자 데이터
data_gid = data[['gid', 'geometry']]

In [12]:
data_gid

,gid,geometry
0,다사579454,"MULTIPOLYGON (((127.02366 37.50691, 127.02365 ..."
1,다사613448,"MULTIPOLYGON (((127.06216 37.50166, 127.06215 ..."
2,다사585452,"MULTIPOLYGON (((127.03046 37.50514, 127.03045 ..."
3,다사583456,"MULTIPOLYGON (((127.02817 37.50874, 127.02817 ..."
4,다사592432,"MULTIPOLYGON (((127.03849 37.48714, 127.03848 ..."
...,...,...
9128,다사559438,"MULTIPOLYGON (((127.00113 37.49240, 127.00112 ..."
9129,다사618387,"MULTIPOLYGON (((127.06813 37.44669, 127.06813 ..."
9130,다사559464,"MULTIPOLYGON (((127.00097 37.51583, 127.00097 ..."
9131,다사585417,"MULTIPOLYGON (((127.03066 37.47359, 127.03065 ..."


# 횡단보도 부착대

In [44]:
data19 = gpd.read_file("output/서초강남횡단보도부착대.geojson")

,STAT_CDE,MGRNU,A062_KND_C,SPRT_STD_C,EVE_CDE,SPRT_HOTDI,CSS_CDE,ESB_YMD,CAE_YMD,OD_PE_CDE,...,field_66,field_67,field_68,field_69,field_70,field_71,field_72,field_73,gid,geometry
0,1,02-0000178068,1,6,1,2,7850.0,20131229,20131229,410.0,...,1),MDSYS.SDO_ORDINATE_ARRAY(204221.148207307,544841.555069,204219.603122,544836.799786406)),252.0,2,0.0,ㅼ궗600449,MULTIPOINT (127.04774 37.50303)
1,1,02-0000178067,1,7,1,2,7850.0,20131229,20131229,410.0,...,1),MDSYS.SDO_ORDINATE_ARRAY(204212.929621534,544827.997402,204217.657214,544829.625242717)),19.0,2,0.0,ㅼ궗600449,MULTIPOINT (127.04765 37.50291)
2,1,02-0000178067,1,7,1,2,7850.0,20131229,20131229,410.0,...,1),MDSYS.SDO_ORDINATE_ARRAY(204212.929621534,544827.997402,204211.219521,544832.695865049)),110.0,2,0.0,ㅼ궗600449,MULTIPOINT (127.04765 37.50291)
3,1,02-0000109590,1,7,1,2,288.0,20131220,20131220,410.0,...,1),MDSYS.SDO_ORDINATE_ARRAY(205374.987783456,544360.323503,205374.986073,544360.328201166)),110.0,1,0.0,ㅼ궗611444,MULTIPOINT (127.06078 37.49869)
4,1,02-0000109590,1,7,1,2,288.0,20131220,20131220,410.0,...,1),MDSYS.SDO_ORDINATE_ARRAY(205374.987783456,544360.323503,205371.045736,544370.592887394)),111.0,3,0.0,ㅼ궗611444,MULTIPOINT (127.06078 37.49869)


In [45]:
gid_list = []
for i in tqdm(range(data19.shape[0])):
    point = data19['geometry'].iloc[i]
    if data_gid['geometry'].contains(point).sum() == 0:
        g1 = "Na"
    else :
        g = data_gid['gid'].loc[data_gid['geometry'].contains(point)]
        g1 = g.iloc[0]
    gid_list.append(g1)

100%|██| 5345/5345 [10:08<00:00,  8.79it/s]


In [46]:
data19['gid'] = gid_list
data19['TL_cnt']=1
result19 = data19[['TL_cnt', 'gid']].groupby(['gid']).sum()
result19.reset_index(inplace=True)
data19 =result19
data19.head()

,gid,TL_cnt
0,다사541420,21
1,다사541421,1
2,다사541424,6
3,다사541425,4
4,다사541427,5


In [47]:
data19

,gid,TL_cnt
0,다사541420,21
1,다사541421,1
2,다사541424,6
3,다사541425,4
4,다사541427,5
...,...,...
1290,다사663411,2
1291,다사663413,2
1292,다사664409,3
1293,다사664410,2


In [48]:
data19.to_csv("output/서초강남횡단보도부착대전처리완료.csv")

# 버스정류장 전처리

In [51]:
data1 = pd.read_csv("output/강남구서초구정류장저녁승차.csv")
data1

,ARS-ID,표준ID,정류소명,X좌표,Y좌표,17시승차총승객수,18시승차총승객수,19시승차총승객수,20시승차총승객수
0,22001,121000001,교육개발원입구,127.038342,37.478514,157.025210,203.605042,112.638655,95.067227
1,22002,121000002,교육개발원입구,127.038676,37.477767,409.878788,659.681818,270.151515,166.909091
2,22003,121000003,양재역.서초문화예술회관,127.035847,37.482522,330.729167,466.736111,386.590278,309.013889
3,22004,121000004,양재역.서초문화예술회관,127.036825,37.481871,271.461538,333.762821,215.237179,166.044872
4,22005,121000005,뱅뱅사거리,127.032367,37.487592,163.512315,203.635468,134.817734,112.167488
...,...,...,...,...,...,...,...,...,...
1161,23972,122900097,수서교회.수서동성당앞,127.094743,37.483491,62.666667,55.416667,15.500000,17.166667
1162,23981,122000286,삼성서울병원사거리,127.082862,37.488968,307.250000,151.916667,92.500000,71.416667
1163,23993,122900073,강남신동아파밀리에2단지,127.101538,37.464601,16.833333,20.041667,20.291667,19.791667
1164,23994,122000287,일원주유소.삼성서울병원후문,127.087681,37.490657,511.419355,324.870968,190.741935,155.516129


In [52]:
data1['bus_cnt']=1
data1['X좌표'] = data1['X좌표'].astype(float)
data1['Y좌표'] = data1['Y좌표'].astype(float)
data1['point'] = data1.apply(lambda row : Point([row['X좌표'], row['Y좌표']]), axis=1)
data1 = gpd.GeoDataFrame(data1, geometry='point')

In [53]:
data1

,ARS-ID,표준ID,정류소명,X좌표,Y좌표,17시승차총승객수,18시승차총승객수,19시승차총승객수,20시승차총승객수,bus_cnt,point
0,22001,121000001,교육개발원입구,127.038342,37.478514,157.025210,203.605042,112.638655,95.067227,1,POINT (127.03834 37.47851)
1,22002,121000002,교육개발원입구,127.038676,37.477767,409.878788,659.681818,270.151515,166.909091,1,POINT (127.03868 37.47777)
2,22003,121000003,양재역.서초문화예술회관,127.035847,37.482522,330.729167,466.736111,386.590278,309.013889,1,POINT (127.03585 37.48252)
3,22004,121000004,양재역.서초문화예술회관,127.036825,37.481871,271.461538,333.762821,215.237179,166.044872,1,POINT (127.03683 37.48187)
4,22005,121000005,뱅뱅사거리,127.032367,37.487592,163.512315,203.635468,134.817734,112.167488,1,POINT (127.03237 37.48759)
...,...,...,...,...,...,...,...,...,...,...,...
1161,23972,122900097,수서교회.수서동성당앞,127.094743,37.483491,62.666667,55.416667,15.500000,17.166667,1,POINT (127.09474 37.48349)
1162,23981,122000286,삼성서울병원사거리,127.082862,37.488968,307.250000,151.916667,92.500000,71.416667,1,POINT (127.08286 37.48897)
1163,23993,122900073,강남신동아파밀리에2단지,127.101538,37.464601,16.833333,20.041667,20.291667,19.791667,1,POINT (127.10154 37.46460)
1164,23994,122000287,일원주유소.삼성서울병원후문,127.087681,37.490657,511.419355,324.870968,190.741935,155.516129,1,POINT (127.08768 37.49066)


In [54]:
gid_list = []
for i in tqdm(range(data1.shape[0])):
    point = data1['point'].iloc[i]
    if data_gid['geometry'].contains(point).sum() == 0:
        g1 = "Na"
    else :
        g = data_gid['gid'].loc[data_gid['geometry'].contains(point)]
        g1 = g.iloc[0]
    gid_list.append(g1)

100%|██| 1166/1166 [02:23<00:00,  8.15it/s]


In [55]:
data1['gid'] = gid_list
data1['TL_cnt']=1
result1 = data1[['TL_cnt', 'gid']].groupby(['gid']).sum()
result1.reset_index(inplace=True)
data1 =result1


In [56]:
data1

,gid,TL_cnt
0,Na,1
1,다사540452,2
2,다사541419,1
3,다사541421,1
4,다사541423,1
...,...,...
774,다사655420,2
775,다사658421,2
776,다사660408,1
777,다사662409,1


In [60]:
data1[data1["gid"] == "Na"]

,gid,TL_cnt
0,Na,1


In [61]:
data2 = data1.drop(index=0, axis=0)

In [62]:
data2

,gid,TL_cnt
1,다사540452,2
2,다사541419,1
3,다사541421,1
4,다사541423,1
5,다사542418,1
...,...,...
774,다사655420,2
775,다사658421,2
776,다사660408,1
777,다사662409,1


In [63]:
data2.to_csv("output/서초강남버스정류장전처리완료.csv")

# 지하철 역 전처리

In [65]:
data3 = pd.read_csv("output/서초강남지하철승하차.csv")
data3

,Unnamed: 0,Unnamed: 0.1,역이름,위도,경도,04시-05시 승차인원,04시-05시 하차인원,05시-06시 승차인원,05시-06시 하차인원,06시-07시 승차인원,...,23시-24시 승차인원,23시-24시 하차인원,00시-01시 승차인원,00시-01시 하차인원,01시-02시 승차인원,01시-02시 하차인원,02시-03시 승차인원,02시-03시 하차인원,03시-04시 승차인원,03시-04시 하차인원
0,5,5,강남구청역,37.517201,127.041287,8,0,2319,2329,5879,...,5963,4919,31,720,0,0,0,0,0,0
1,6,6,강남역,37.498085,127.027978,85,6,5464,10853,15861,...,24062,12861,7,4,0,0,0,0,0,0
2,12,12,개포동역,37.489132,127.066103,2,0,687,1011,3267,...,650,2034,5,58,0,0,0,0,0,0
3,20,20,고속터미널역,37.504540,127.005135,162,8,7178,6966,14260,...,19976,14774,18,387,0,0,0,0,0,0
4,28,28,교대역,37.492736,127.013863,43,0,3120,7490,14922,...,9727,8907,2,38,0,1,0,1,0,0
5,31,31,구룡역,37.487027,127.059475,0,0,360,829,1312,...,170,936,8,27,0,0,0,0,0,0
6,32,32,구반포역,37.501422,126.987447,0,0,516,236,1717,...,376,1491,0,45,0,0,0,0,0,0
7,37,37,국회의사당역,37.528136,126.917821,3,0,607,1731,1777,...,2165,1061,0,0,0,0,0,0,0,0
8,48,48,남부터미널역,37.485185,127.016191,10,0,2452,1284,8012,...,4321,6529,4,28,1,3,1,0,0,0
9,51,51,남태령역,37.464246,126.989081,46,0,453,126,824,...,90,384,0,31,0,0,0,0,0,0


In [66]:
data3 = data3[["역이름","위도","경도"]]

In [70]:
data3

,역이름,위도,경도,sub_cnt,point
0,강남구청역,37.517201,127.041287,1,POINT (127.04129 37.51720)
1,강남역,37.498085,127.027978,1,POINT (127.02798 37.49809)
2,개포동역,37.489132,127.066103,1,POINT (127.06610 37.48913)
3,고속터미널역,37.504540,127.005135,1,POINT (127.00513 37.50454)
4,교대역,37.492736,127.013863,1,POINT (127.01386 37.49274)
5,구룡역,37.487027,127.059475,1,POINT (127.05947 37.48703)
6,구반포역,37.501422,126.987447,1,POINT (126.98745 37.50142)
7,국회의사당역,37.528136,126.917821,1,POINT (126.91782 37.52814)
8,남부터미널역,37.485185,127.016191,1,POINT (127.01619 37.48519)
9,남태령역,37.464246,126.989081,1,POINT (126.98908 37.46425)


In [68]:
data3['sub_cnt']=1
data3['경도'] = data3['경도'].astype(float)
data3['위도'] = data3['위도'].astype(float)
data3['point'] = data3.apply(lambda row : Point([row['경도'], row['위도']]), axis=1)
data3 = gpd.GeoDataFrame(data3, geometry='point')

<ipython-input-68-9f078b38371f>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data3['sub_cnt']=1
<ipython-input-68-9f078b38371f>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data3['경도'] = data3['경도'].astype(float)
<ipython-input-68-9f078b38371f>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html

In [69]:
data3

,역이름,위도,경도,sub_cnt,point
0,강남구청역,37.517201,127.041287,1,POINT (127.04129 37.51720)
1,강남역,37.498085,127.027978,1,POINT (127.02798 37.49809)
2,개포동역,37.489132,127.066103,1,POINT (127.06610 37.48913)
3,고속터미널역,37.504540,127.005135,1,POINT (127.00513 37.50454)
4,교대역,37.492736,127.013863,1,POINT (127.01386 37.49274)
5,구룡역,37.487027,127.059475,1,POINT (127.05947 37.48703)
6,구반포역,37.501422,126.987447,1,POINT (126.98745 37.50142)
7,국회의사당역,37.528136,126.917821,1,POINT (126.91782 37.52814)
8,남부터미널역,37.485185,127.016191,1,POINT (127.01619 37.48519)
9,남태령역,37.464246,126.989081,1,POINT (126.98908 37.46425)


In [71]:
gid_list = []
for i in tqdm(range(data3.shape[0])):
    point = data3['point'].iloc[i]
    if data_gid['geometry'].contains(point).sum() == 0:
        g1 = "Na"
    else :
        g = data_gid['gid'].loc[data_gid['geometry'].contains(point)]
        g1 = g.iloc[0]
    gid_list.append(g1)

100%|██████| 47/47 [00:06<00:00,  7.40it/s]


In [72]:
data3['gid'] = gid_list
data3['TL_cnt']=1
result3 = data3[['TL_cnt', 'gid']].groupby(['gid']).sum()
result3.reset_index(inplace=True)
data3 =result3


In [73]:
data3

,gid,TL_cnt
0,Na,3
1,다사540449,1
2,다사541420,1
3,다사542431,2
4,다사546448,1
5,다사548406,1
6,다사552432,1
7,다사554450,1
8,다사555425,1
9,다사562451,1


In [74]:
data3[data3["gid"] == "Na"]

,gid,TL_cnt
0,Na,3


In [75]:
data4 = data3.drop(index=0, axis=0)

In [76]:
data4

,gid,TL_cnt
1,다사540449,1
2,다사541420,1
3,다사542431,2
4,다사546448,1
5,다사548406,1
6,다사552432,1
7,다사554450,1
8,다사555425,1
9,다사562451,1
10,다사564437,1


In [77]:
data4.to_csv("output/서초강남지하철역전처리완료.csv")

# 상권 전처리

In [24]:
store = pd.read_csv("output/서초강남상권.csv")
store

,Unnamed: 0,상가업소번호,상호명,지점명,상권업종대분류코드,상권업종대분류명,상권업종중분류코드,상권업종중분류명,상권업종소분류코드,상권업종소분류명,...,구우편번호,신우편번호,동정보,층정보,호정보,전화번호,경도,위도,상권번호,데이터기준일자
0,0,17174175,비지트,NaN,Q,음식,Q01,한식,Q01A01,한식/백반/한정식,...,137837,6572.0,NaN,1,102,NaN,126.991394,37.488375,NaN,2021-01-09
1,18,17174636,호랑이가만든족발,NaN,Q,음식,Q01,한식,Q01A08,족발/보쌈전문,...,135513,6148.0,NaN,NaN,NaN,02-6465-1314,127.044182,37.504996,1943.0,2021-01-09
2,19,17174040,다향,NaN,Q,음식,Q01,한식,Q01A01,한식/백반/한정식,...,137869,6709.0,NaN,1,NaN,02-583-5658,127.009382,37.483436,NaN,2020-11-27
3,20,25530299,고향생막걸리,NaN,Q,음식,Q09,유흥주점,Q09A04,민속주점,...,135894,6023.0,NaN,1,NaN,02-544-8819,127.031012,37.528073,2053.0,2017-08-27
4,23,22795034,든든한우家,NaN,Q,음식,Q01,한식,Q01A01,한식/백반/한정식,...,137070,6652.0,NaN,2,NaN,02-586-4000,127.012363,37.486335,1845.0,2017-09-23
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47854,316900,17456528,크린스퀘어,NaN,F,생활서비스,F02,세탁/가사서비스,F02A01,세탁소/빨래방,...,135877,6158.0,101,1,B110,NaN,127.055465,37.508079,NaN,2021-02-11
47855,316902,17431606,안젤라화이트,NaN,F,생활서비스,F01,이/미용/건강,F01A03,비만/피부관리,...,137817,6677.0,NaN,2,201,NaN,126.982924,37.482598,NaN,2021-01-11
47856,316913,17456260,서울방역공사,NaN,F,생활서비스,F02,세탁/가사서비스,F02A05,청소/소독,...,137843,6682.0,NaN,1,NaN,NaN,126.993660,37.484673,NaN,2021-02-11
47857,316917,17446911,서래와규짱,서래본점,Q,음식,Q01,한식,Q01A01,한식/백반/한정식,...,137831,6585.0,NaN,NaN,NaN,NaN,126.991262,37.492275,NaN,2021-01-26


In [91]:
store.상권업종대분류명.unique()

array(['음식', '부동산', '소매', '생활서비스', '관광/여가/오락', '학문/교육', '숙박', '스포츠'],
      dtype=object)

In [110]:
contains_대분류 = store['상권업종대분류명'].str.contains("음식")

In [111]:
서초강남음식점 = store[contains_대분류]

In [112]:
서초강남음식점.to_csv("output/상업대분류/서초강남음식점.csv")

In [25]:
contains_대분류 = store['상권업종대분류명'].str.contains("부동산")

In [26]:
서초강남부동산 = store[contains_대분류]

In [27]:
서초강남부동산

,Unnamed: 0,상가업소번호,상호명,지점명,상권업종대분류코드,상권업종대분류명,상권업종중분류코드,상권업종중분류명,상권업종소분류코드,상권업종소분류명,...,구우편번호,신우편번호,동정보,층정보,호정보,전화번호,경도,위도,상권번호,데이터기준일자
5,37,25545908,주공부동산,NaN,L,부동산,L01,부동산중개,L01A01,부동산중개,...,135993,6330.0,NaN,1,NaN,NaN,127.070856,37.489134,NaN,2017-10-30
13,67,25448752,럭키공인,NaN,L,부동산,L01,부동산중개,L01A01,부동산중개,...,135270,6264.0,NaN,1,NaN,02-2057-8945,127.035670,37.487032,1839.0,2016-01-26
30,166,20968054,강변공인중개사사무소,NaN,L,부동산,L01,부동산중개,L01A01,부동산중개,...,137907,6517.0,NaN,NaN,NaN,02-536-5555,127.012702,37.518127,NaN,2017-09-30
39,210,15598991,로즈모닝공인중개사사무소,NaN,L,부동산,L01,부동산중개,L01A01,부동산중개,...,135744,6367.0,NaN,NaN,102,02-3413-1234,127.103094,37.487311,1848.0,2017-10-02
46,254,15612953,반포자이이화공인중개사사무소,NaN,L,부동산,L01,부동산중개,L01A01,부동산중개,...,137041,6544.0,NaN,NaN,103,02-537-4111,127.009861,37.507110,1940.0,2017-10-02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47543,314864,17454228,영조개발,NaN,L,부동산,L01,부동산중개,L01A01,부동산중개,...,135272,6276.0,NaN,1,110,NaN,127.052707,37.495361,NaN,2021-02-08
47624,315504,17356195,에스케이광개토공인중개사사무소,NaN,L,부동산,L01,부동산중개,L01A01,부동산중개,...,135936,6242.0,NaN,1,101,NaN,127.031226,37.494024,1893.0,2021-01-19
47721,316186,17443509,에버그린감정평가사사무소,NaN,L,부동산,L04,평가/개발/관리,L04A04,부동산감정평가,...,137907,6517.0,101,10,1003,NaN,127.012390,37.518611,NaN,2021-01-19
47749,316354,17353689,개포나무공인중개사사무소,NaN,L,부동산,L01,부동산중개,L01A01,부동산중개,...,135807,6319.0,상가,2,201,NaN,127.056077,37.484620,NaN,2021-01-19


In [28]:
서초강남부동산.to_csv("output/상업대분류/서초강남부동산.csv")

In [98]:
contains_대분류 = store['상권업종대분류명'].str.contains("소매")

In [99]:
서초강남소매점 = store[contains_대분류]

In [114]:
서초강남소매점.to_csv("output/상업대분류/서초강남소매점.csv")

In [100]:
contains_대분류 = store['상권업종대분류명'].str.contains("생활서비스")

In [101]:
서초강남생활서비스 = store[contains_대분류]

In [115]:
서초강남생활서비스.to_csv("output/상업대분류/서초강남생활서비스.csv")

In [102]:
contains_대분류 = store['상권업종대분류명'].str.contains("관광/여가/오락")

In [103]:
서초강남관광여가오락 = store[contains_대분류]

In [116]:
서초강남관광여가오락.to_csv("output/상업대분류/서초강남관광여가오락.csv")

In [104]:
contains_대분류 = store['상권업종대분류명'].str.contains("학문/교육")

In [105]:
서초강남학문교육 = store[contains_대분류]

In [117]:
서초강남학문교육.to_csv("output/상업대분류/서초강남학문교육.csv")

In [106]:
contains_대분류 = store['상권업종대분류명'].str.contains("숙박")

In [107]:
서초강남숙박 = store[contains_대분류]

In [118]:
서초강남숙박.to_csv("output/상업대분류/서초강남숙박.csv")

In [108]:
contains_대분류 = store['상권업종대분류명'].str.contains("스포츠")

In [109]:
서초강남스포츠 = store[contains_대분류]

In [119]:
서초강남스포츠.to_csv("output/상업대분류/서초강남스포츠.csv")

### '음식', '부동산', '소매', '생활서비스', '관광/여가/오락', '학문/교육', '숙박', '스포츠' : 상권업종대분류명으로 나눈 후에 전처리 

In [7]:
data3 = pd.read_csv("output/상업대분류/서초강남음식점.csv")
data3

,Unnamed: 0,Unnamed: 0.1,상가업소번호,상호명,지점명,상권업종대분류코드,상권업종대분류명,상권업종중분류코드,상권업종중분류명,상권업종소분류코드,...,구우편번호,신우편번호,동정보,층정보,호정보,전화번호,경도,위도,상권번호,데이터기준일자
0,0,0,17174175,비지트,NaN,Q,음식,Q01,한식,Q01A01,...,137837,6572.0,NaN,1,102,NaN,126.991394,37.488375,NaN,2021-01-09
1,1,18,17174636,호랑이가만든족발,NaN,Q,음식,Q01,한식,Q01A08,...,135513,6148.0,NaN,NaN,NaN,02-6465-1314,127.044182,37.504996,1943.0,2021-01-09
2,2,19,17174040,다향,NaN,Q,음식,Q01,한식,Q01A01,...,137869,6709.0,NaN,1,NaN,02-583-5658,127.009382,37.483436,NaN,2020-11-27
3,3,20,25530299,고향생막걸리,NaN,Q,음식,Q09,유흥주점,Q09A04,...,135894,6023.0,NaN,1,NaN,02-544-8819,127.031012,37.528073,2053.0,2017-08-27
4,4,23,22795034,든든한우家,NaN,Q,음식,Q01,한식,Q01A01,...,137070,6652.0,NaN,2,NaN,02-586-4000,127.012363,37.486335,1845.0,2017-09-23
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19377,47850,316880,17458834,배달육회육회한가득신사,논현점,Q,음식,Q01,한식,Q01A01,...,135814,6043.0,201,1,7,NaN,127.022889,37.515058,1993.0,2021-02-26
19378,47851,316881,17456030,신차우,NaN,Q,음식,Q01,한식,Q01A01,...,135830,6099.0,NaN,NaN,NaN,NaN,127.040489,37.516076,NaN,2021-02-11
19379,47853,316894,17456240,데이브레드한시적,NaN,Q,음식,Q08,제과제빵떡케익,Q08A01,...,135777,6194.0,NaN,1,NaN,NaN,127.056116,37.505804,1932.0,2021-02-11
19380,47857,316917,17446911,서래와규짱,서래본점,Q,음식,Q01,한식,Q01A01,...,137831,6585.0,NaN,NaN,NaN,NaN,126.991262,37.492275,NaN,2021-01-26


In [8]:
data3 = data3[["상권업종대분류명","위도","경도"]]

In [9]:
data3

,상권업종대분류명,위도,경도
0,음식,37.488375,126.991394
1,음식,37.504996,127.044182
2,음식,37.483436,127.009382
3,음식,37.528073,127.031012
4,음식,37.486335,127.012363
...,...,...,...
19377,음식,37.515058,127.022889
19378,음식,37.516076,127.040489
19379,음식,37.505804,127.056116
19380,음식,37.492275,126.991262


In [10]:
data3['store_cnt']=1
data3['경도'] = data3['경도'].astype(float)
data3['위도'] = data3['위도'].astype(float)
data3['point'] = data3.apply(lambda row : Point([row['경도'], row['위도']]), axis=1)
data3 = gpd.GeoDataFrame(data3, geometry='point')

<ipython-input-10-9ea83613d36b>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data3['store_cnt']=1
<ipython-input-10-9ea83613d36b>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data3['경도'] = data3['경도'].astype(float)
<ipython-input-10-9ea83613d36b>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.ht

In [11]:
data3

,상권업종대분류명,위도,경도,store_cnt,point
0,음식,37.488375,126.991394,1,POINT (126.99139 37.48837)
1,음식,37.504996,127.044182,1,POINT (127.04418 37.50500)
2,음식,37.483436,127.009382,1,POINT (127.00938 37.48344)
3,음식,37.528073,127.031012,1,POINT (127.03101 37.52807)
4,음식,37.486335,127.012363,1,POINT (127.01236 37.48634)
...,...,...,...,...,...
19377,음식,37.515058,127.022889,1,POINT (127.02289 37.51506)
19378,음식,37.516076,127.040489,1,POINT (127.04049 37.51608)
19379,음식,37.505804,127.056116,1,POINT (127.05612 37.50580)
19380,음식,37.492275,126.991262,1,POINT (126.99126 37.49227)


In [12]:
gid_list = []
for i in tqdm(range(data3.shape[0])):
    point = data3['point'].iloc[i]
    if data_gid['geometry'].contains(point).sum() == 0:
        g1 = "Na"
    else :
        g = data_gid['gid'].loc[data_gid['geometry'].contains(point)]
        g1 = g.iloc[0]
    gid_list.append(g1)

100%|████████████████████████████████████████████████████████████████████████████| 19382/19382 [40:47<00:00,  7.92it/s]


In [13]:
data3['gid'] = gid_list
data3['TL_cnt']=1
result3 = data3[['TL_cnt', 'gid']].groupby(['gid']).sum()
result3.reset_index(inplace=True)
data3 =result3


In [14]:
data3

,gid,TL_cnt
0,다사541452,6
1,다사542417,1
2,다사542418,1
3,다사542420,2
4,다사542421,18
...,...,...
2454,다사656419,1
2455,다사658417,3
2456,다사659417,2
2457,다사662413,2


In [16]:
data3.to_csv("output/서초강남음식점전처리완료.csv")

## '부동산'

In [29]:
data3 = pd.read_csv("output/상업대분류/서초강남부동산.csv")
data3

,Unnamed: 0,Unnamed: 0.1,상가업소번호,상호명,지점명,상권업종대분류코드,상권업종대분류명,상권업종중분류코드,상권업종중분류명,상권업종소분류코드,...,구우편번호,신우편번호,동정보,층정보,호정보,전화번호,경도,위도,상권번호,데이터기준일자
0,5,37,25545908,주공부동산,NaN,L,부동산,L01,부동산중개,L01A01,...,135993,6330.0,NaN,1,NaN,NaN,127.070856,37.489134,NaN,2017-10-30
1,13,67,25448752,럭키공인,NaN,L,부동산,L01,부동산중개,L01A01,...,135270,6264.0,NaN,1,NaN,02-2057-8945,127.035670,37.487032,1839.0,2016-01-26
2,30,166,20968054,강변공인중개사사무소,NaN,L,부동산,L01,부동산중개,L01A01,...,137907,6517.0,NaN,NaN,NaN,02-536-5555,127.012702,37.518127,NaN,2017-09-30
3,39,210,15598991,로즈모닝공인중개사사무소,NaN,L,부동산,L01,부동산중개,L01A01,...,135744,6367.0,NaN,NaN,102,02-3413-1234,127.103094,37.487311,1848.0,2017-10-02
4,46,254,15612953,반포자이이화공인중개사사무소,NaN,L,부동산,L01,부동산중개,L01A01,...,137041,6544.0,NaN,NaN,103,02-537-4111,127.009861,37.507110,1940.0,2017-10-02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2596,47543,314864,17454228,영조개발,NaN,L,부동산,L01,부동산중개,L01A01,...,135272,6276.0,NaN,1,110,NaN,127.052707,37.495361,NaN,2021-02-08
2597,47624,315504,17356195,에스케이광개토공인중개사사무소,NaN,L,부동산,L01,부동산중개,L01A01,...,135936,6242.0,NaN,1,101,NaN,127.031226,37.494024,1893.0,2021-01-19
2598,47721,316186,17443509,에버그린감정평가사사무소,NaN,L,부동산,L04,평가/개발/관리,L04A04,...,137907,6517.0,101,10,1003,NaN,127.012390,37.518611,NaN,2021-01-19
2599,47749,316354,17353689,개포나무공인중개사사무소,NaN,L,부동산,L01,부동산중개,L01A01,...,135807,6319.0,상가,2,201,NaN,127.056077,37.484620,NaN,2021-01-19


In [18]:
data3 = data3[["상권업종대분류명","위도","경도"]]

In [19]:
data3

,상권업종대분류명,위도,경도
0,음식,37.488375,126.991394
1,음식,37.504996,127.044182
2,음식,37.483436,127.009382
3,음식,37.528073,127.031012
4,음식,37.486335,127.012363
...,...,...,...
19377,음식,37.515058,127.022889
19378,음식,37.516076,127.040489
19379,음식,37.505804,127.056116
19380,음식,37.492275,126.991262


In [30]:
data3['store_cnt']=1
data3['경도'] = data3['경도'].astype(float)
data3['위도'] = data3['위도'].astype(float)
data3['point'] = data3.apply(lambda row : Point([row['경도'], row['위도']]), axis=1)
data3 = gpd.GeoDataFrame(data3, geometry='point')

In [31]:
data3

,Unnamed: 0,Unnamed: 0.1,상가업소번호,상호명,지점명,상권업종대분류코드,상권업종대분류명,상권업종중분류코드,상권업종중분류명,상권업종소분류코드,...,동정보,층정보,호정보,전화번호,경도,위도,상권번호,데이터기준일자,store_cnt,point
0,5,37,25545908,주공부동산,NaN,L,부동산,L01,부동산중개,L01A01,...,NaN,1,NaN,NaN,127.070856,37.489134,NaN,2017-10-30,1,POINT (127.07086 37.48913)
1,13,67,25448752,럭키공인,NaN,L,부동산,L01,부동산중개,L01A01,...,NaN,1,NaN,02-2057-8945,127.035670,37.487032,1839.0,2016-01-26,1,POINT (127.03567 37.48703)
2,30,166,20968054,강변공인중개사사무소,NaN,L,부동산,L01,부동산중개,L01A01,...,NaN,NaN,NaN,02-536-5555,127.012702,37.518127,NaN,2017-09-30,1,POINT (127.01270 37.51813)
3,39,210,15598991,로즈모닝공인중개사사무소,NaN,L,부동산,L01,부동산중개,L01A01,...,NaN,NaN,102,02-3413-1234,127.103094,37.487311,1848.0,2017-10-02,1,POINT (127.10309 37.48731)
4,46,254,15612953,반포자이이화공인중개사사무소,NaN,L,부동산,L01,부동산중개,L01A01,...,NaN,NaN,103,02-537-4111,127.009861,37.507110,1940.0,2017-10-02,1,POINT (127.00986 37.50711)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2596,47543,314864,17454228,영조개발,NaN,L,부동산,L01,부동산중개,L01A01,...,NaN,1,110,NaN,127.052707,37.495361,NaN,2021-02-08,1,POINT (127.05271 37.49536)
2597,47624,315504,17356195,에스케이광개토공인중개사사무소,NaN,L,부동산,L01,부동산중개,L01A01,...,NaN,1,101,NaN,127.031226,37.494024,1893.0,2021-01-19,1,POINT (127.03123 37.49402)
2598,47721,316186,17443509,에버그린감정평가사사무소,NaN,L,부동산,L04,평가/개발/관리,L04A04,...,101,10,1003,NaN,127.012390,37.518611,NaN,2021-01-19,1,POINT (127.01239 37.51861)
2599,47749,316354,17353689,개포나무공인중개사사무소,NaN,L,부동산,L01,부동산중개,L01A01,...,상가,2,201,NaN,127.056077,37.484620,NaN,2021-01-19,1,POINT (127.05608 37.48462)


In [32]:
gid_list = []
for i in tqdm(range(data3.shape[0])):
    point = data3['point'].iloc[i]
    if data_gid['geometry'].contains(point).sum() == 0:
        g1 = "Na"
    else :
        g = data_gid['gid'].loc[data_gid['geometry'].contains(point)]
        g1 = g.iloc[0]
    gid_list.append(g1)

100%|██████████████████████████████████████████████████████████████████████████████| 2601/2601 [06:06<00:00,  7.10it/s]


In [33]:
data3['gid'] = gid_list
data3['TL_cnt']=1
result3 = data3[['TL_cnt', 'gid']].groupby(['gid']).sum()
result3.reset_index(inplace=True)
data3 =result3


In [34]:
data3

,gid,TL_cnt
0,다사542417,2
1,다사542418,1
2,다사542424,1
3,다사542425,1
4,다사542427,4
...,...,...
1228,다사656419,2
1229,다사662413,2
1230,다사663408,1
1231,다사663409,1


In [35]:
data3.to_csv("output/서초강남부동산전처리완료.csv")

##  '소매'

In [44]:
data3 = pd.read_csv("output/상업대분류/서초강남소매점.csv")
data3

,Unnamed: 0,Unnamed: 0.1,상가업소번호,상호명,지점명,상권업종대분류코드,상권업종대분류명,상권업종중분류코드,상권업종중분류명,상권업종소분류코드,...,구우편번호,신우편번호,동정보,층정보,호정보,전화번호,경도,위도,상권번호,데이터기준일자
0,6,38,20906101,닥터드레스,NaN,D,소매,D05,의복의류,D05A09,...,135280,6207.0,NaN,NaN,NaN,02-123-4567,127.054650,37.497359,1906.0,2016-01-26
1,7,45,24477849,피터팬플라워,NaN,D,소매,D02,선물/팬시/기념품,D02A02,...,137040,6545.0,NaN,3,NaN,02-595-6912,127.006816,37.506427,1940.0,2017-09-13
2,8,48,28500399,캐논비즈니스프라자전시장,비즈니스프라자전시장,D,소매,D08,사무/문구/컴퓨터,D08A02,...,135090,6173.0,NaN,1,NaN,02-3456-0691,127.063987,37.509507,NaN,2017-08-23
3,14,79,24039016,김성자콜렉션,NaN,D,소매,D05,의복의류,D05A18,...,137040,6545.0,NaN,4,NaN,02-533-1943,127.006816,37.506427,1940.0,2016-12-21
4,15,100,24041982,더쉬스튜디오,NaN,D,소매,D18,사진/광학/정밀기기소매,D18A03,...,135010,6059.0,하,1,NaN,02-542-4933,127.040164,37.518462,NaN,2016-01-27
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12267,47717,316152,17439856,보네르스포사,NaN,D,소매,D05,의복의류,D05A12,...,135820,6059.0,NaN,NaN,NaN,02-6498-6006,127.039878,37.518721,NaN,2021-01-14
12268,47747,316340,17435780,목서플라워,NaN,D,소매,D02,선물/팬시/기념품,D02A02,...,135917,6149.0,NaN,NaN,NaN,02-6081-2889,127.046697,37.506458,1943.0,2021-01-14
12269,47748,316342,17437751,플라우드플라워,NaN,D,소매,D02,선물/팬시/기념품,D02A02,...,137809,6540.0,NaN,NaN,NaN,NaN,127.021260,37.507269,1960.0,2021-01-14
12270,47839,316811,17371246,세븐일레븐,강남도곡로점,D,소매,D03,종합소매점,D03A01,...,135857,6261.0,NaN,1,NaN,NaN,127.039110,37.491277,NaN,2021-01-13


In [45]:
data3 = data3[["상권업종대분류명","위도","경도"]]

In [46]:
data3

,상권업종대분류명,위도,경도
0,소매,37.497359,127.054650
1,소매,37.506427,127.006816
2,소매,37.509507,127.063987
3,소매,37.506427,127.006816
4,소매,37.518462,127.040164
...,...,...,...
12267,소매,37.518721,127.039878
12268,소매,37.506458,127.046697
12269,소매,37.507269,127.021260
12270,소매,37.491277,127.039110


In [47]:
data3['store_cnt']=1
data3['경도'] = data3['경도'].astype(float)
data3['위도'] = data3['위도'].astype(float)
data3['point'] = data3.apply(lambda row : Point([row['경도'], row['위도']]), axis=1)
data3 = gpd.GeoDataFrame(data3, geometry='point')

<ipython-input-47-9ea83613d36b>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data3['store_cnt']=1
<ipython-input-47-9ea83613d36b>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data3['경도'] = data3['경도'].astype(float)
<ipython-input-47-9ea83613d36b>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.ht

In [48]:
data3

,상권업종대분류명,위도,경도,store_cnt,point
0,소매,37.497359,127.054650,1,POINT (127.05465 37.49736)
1,소매,37.506427,127.006816,1,POINT (127.00682 37.50643)
2,소매,37.509507,127.063987,1,POINT (127.06399 37.50951)
3,소매,37.506427,127.006816,1,POINT (127.00682 37.50643)
4,소매,37.518462,127.040164,1,POINT (127.04016 37.51846)
...,...,...,...,...,...
12267,소매,37.518721,127.039878,1,POINT (127.03988 37.51872)
12268,소매,37.506458,127.046697,1,POINT (127.04670 37.50646)
12269,소매,37.507269,127.021260,1,POINT (127.02126 37.50727)
12270,소매,37.491277,127.039110,1,POINT (127.03911 37.49128)


In [49]:
gid_list = []
for i in tqdm(range(data3.shape[0])):
    point = data3['point'].iloc[i]
    if data_gid['geometry'].contains(point).sum() == 0:
        g1 = "Na"
    else :
        g = data_gid['gid'].loc[data_gid['geometry'].contains(point)]
        g1 = g.iloc[0]
    gid_list.append(g1)

100%|████████████████████████████████████████████████████████████████████████████| 12272/12272 [26:09<00:00,  7.82it/s]


In [50]:
data3['gid'] = gid_list
data3['TL_cnt']=1
result3 = data3[['TL_cnt', 'gid']].groupby(['gid']).sum()
result3.reset_index(inplace=True)
data3 =result3


In [51]:
data3

,gid,TL_cnt
0,다사541452,2
1,다사542417,3
2,다사542421,5
3,다사542422,2
4,다사542424,4
...,...,...
2467,다사656419,3
2468,다사657420,1
2469,다사662413,11
2470,다사663408,1


In [52]:
data3.to_csv("output/서초강남소매점전처리완료.csv")

## '생활서비스'

In [53]:
data3 = pd.read_csv("output/상업대분류/서초강남생활서비스.csv")
data3

,Unnamed: 0,Unnamed: 0.1,상가업소번호,상호명,지점명,상권업종대분류코드,상권업종대분류명,상권업종중분류코드,상권업종중분류명,상권업종소분류코드,...,구우편번호,신우편번호,동정보,층정보,호정보,전화번호,경도,위도,상권번호,데이터기준일자
0,9,51,6112563,소낙스,역삼점,F,생활서비스,F14,자동차/이륜차,F14A04,...,135080,6149.0,NaN,NaN,NaN,02-3288-6300,127.046474,37.507105,1943.0,2016-01-27
1,11,59,21942073,디자인나르세인,NaN,F,생활서비스,F13,기타서비스업,F13A11,...,137060,6560.0,NaN,NaN,NaN,02-336-9877,126.986572,37.492073,NaN,2016-01-26
2,20,123,15570200,이데에,NaN,F,생활서비스,F13,기타서비스업,F13A01,...,137903,6526.0,NaN,4,NaN,02-517-6540,127.018788,37.513817,1984.0,2017-10-01
3,24,142,20949708,크린토피아,대치선경상가점,F,생활서비스,F02,세탁/가사서비스,F02A01,...,135836,6291.0,NaN,NaN,NaN,02-123-4567,127.062981,37.493145,1875.0,2017-02-20
4,35,190,23080605,현대세탁소,NaN,F,생활서비스,F02,세탁/가사서비스,F02A01,...,135080,6137.0,NaN,NaN,NaN,02-569-8890,127.036881,37.507226,1946.0,2016-01-26
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8548,47844,316842,17433587,나,서초구립느티무쉼터내곡느티사우나,F,생활서비스,F09,대중목욕탕/휴게,F09A01,...,137180,6793.0,NaN,1,NaN,NaN,127.050565,37.461788,NaN,2021-01-13
8549,47852,316889,17433065,프롬.린붙임머리,NaN,F,생활서비스,F01,이/미용/건강,F01A01,...,135502,6175.0,NaN,2,204,NaN,127.066750,37.508349,1950.0,2021-01-12
8550,47854,316900,17456528,크린스퀘어,NaN,F,생활서비스,F02,세탁/가사서비스,F02A01,...,135877,6158.0,101,1,B110,NaN,127.055465,37.508079,NaN,2021-02-11
8551,47855,316902,17431606,안젤라화이트,NaN,F,생활서비스,F01,이/미용/건강,F01A03,...,137817,6677.0,NaN,2,201,NaN,126.982924,37.482598,NaN,2021-01-11


In [54]:
data3 = data3[["상권업종대분류명","위도","경도"]]

In [55]:
data3

,상권업종대분류명,위도,경도
0,생활서비스,37.507105,127.046474
1,생활서비스,37.492073,126.986572
2,생활서비스,37.513817,127.018788
3,생활서비스,37.493145,127.062981
4,생활서비스,37.507226,127.036881
...,...,...,...
8548,생활서비스,37.461788,127.050565
8549,생활서비스,37.508349,127.066750
8550,생활서비스,37.508079,127.055465
8551,생활서비스,37.482598,126.982924


In [56]:
data3['store_cnt']=1
data3['경도'] = data3['경도'].astype(float)
data3['위도'] = data3['위도'].astype(float)
data3['point'] = data3.apply(lambda row : Point([row['경도'], row['위도']]), axis=1)
data3 = gpd.GeoDataFrame(data3, geometry='point')

<ipython-input-56-9ea83613d36b>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data3['store_cnt']=1
<ipython-input-56-9ea83613d36b>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data3['경도'] = data3['경도'].astype(float)
<ipython-input-56-9ea83613d36b>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.ht

In [57]:
data3

,상권업종대분류명,위도,경도,store_cnt,point
0,생활서비스,37.507105,127.046474,1,POINT (127.04647 37.50711)
1,생활서비스,37.492073,126.986572,1,POINT (126.98657 37.49207)
2,생활서비스,37.513817,127.018788,1,POINT (127.01879 37.51382)
3,생활서비스,37.493145,127.062981,1,POINT (127.06298 37.49314)
4,생활서비스,37.507226,127.036881,1,POINT (127.03688 37.50723)
...,...,...,...,...,...
8548,생활서비스,37.461788,127.050565,1,POINT (127.05057 37.46179)
8549,생활서비스,37.508349,127.066750,1,POINT (127.06675 37.50835)
8550,생활서비스,37.508079,127.055465,1,POINT (127.05546 37.50808)
8551,생활서비스,37.482598,126.982924,1,POINT (126.98292 37.48260)


In [58]:
gid_list = []
for i in tqdm(range(data3.shape[0])):
    point = data3['point'].iloc[i]
    if data_gid['geometry'].contains(point).sum() == 0:
        g1 = "Na"
    else :
        g = data_gid['gid'].loc[data_gid['geometry'].contains(point)]
        g1 = g.iloc[0]
    gid_list.append(g1)

100%|██████████████████████████████████████████████████████████████████████████████| 8553/8553 [18:54<00:00,  7.54it/s]


In [59]:
data3['gid'] = gid_list
data3['TL_cnt']=1
result3 = data3[['TL_cnt', 'gid']].groupby(['gid']).sum()
result3.reset_index(inplace=True)
data3 =result3

In [60]:
data3

,gid,TL_cnt
0,다사542417,4
1,다사542418,3
2,다사542422,2
3,다사542423,2
4,다사542424,1
...,...,...
2265,다사662413,6
2266,다사663408,5
2267,다사663411,5
2268,다사664410,1


In [61]:
data3.to_csv("output/서초강남생활서비스전처리완료.csv")

## '관광/여가/오락'

In [62]:
data3 = pd.read_csv("output/상업대분류/서초강남관광여가오락.csv")
data3

,Unnamed: 0,Unnamed: 0.1,상가업소번호,상호명,지점명,상권업종대분류코드,상권업종대분류명,상권업종중분류코드,상권업종중분류명,상권업종소분류코드,...,구우편번호,신우편번호,동정보,층정보,호정보,전화번호,경도,위도,상권번호,데이터기준일자
0,10,52,20783574,선경바둑,NaN,N,관광/여가/오락,N01,PC/오락/당구/볼링등,N01A09,...,135896,6017.0,NaN,3,NaN,02-544-3338,127.035280,37.528775,2054.0,2017-08-27
1,43,228,15589624,디발롱스필라테스,NaN,N,관광/여가/오락,N05,요가/단전/마사지,N05A01,...,137903,6526.0,NaN,NaN,303,02-547-3345,127.019521,37.514850,1984.0,2017-10-02
2,78,486,15487768,강남윈드오케스트라,NaN,N,관광/여가/오락,N03,연극/영화/극장,N03A06,...,137877,6643.0,9,NaN,6,NaN,127.017045,37.485410,NaN,2017-09-30
3,130,851,24754323,허브당구장,NaN,N,관광/여가/오락,N01,PC/오락/당구/볼링등,N01A03,...,137130,6779.0,NaN,1,NaN,070-8104-3439,127.040265,37.469075,1760.0,2017-10-30
4,161,1117,25452005,솔리드피씨,NaN,N,관광/여가/오락,N01,PC/오락/당구/볼링등,N01A01,...,137060,6698.0,NaN,1,NaN,02-867-7722,126.988447,37.477960,NaN,2016-01-26
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1016,47235,312884,25405074,대성게임,NaN,N,관광/여가/오락,N01,PC/오락/당구/볼링등,N01A01,...,137797,6509.0,NaN,2,NaN,NaN,127.004451,37.511651,NaN,2016-01-26
1017,47327,313503,17356262,휠라,테스라운지,N,관광/여가/오락,N05,요가/단전/마사지,N05A01,...,135888,6028.0,NaN,NaN,405,NaN,127.019997,37.522567,2045.0,2021-01-19
1018,47328,313507,17350061,아메리카요가,삼성중앙역점,N,관광/여가/오락,N05,요가/단전/마사지,N05A01,...,135868,6096.0,NaN,4,NaN,NaN,127.050613,37.512518,NaN,2021-01-19
1019,47751,316360,17442795,크로스아동문화예술센터,NaN,N,관광/여가/오락,N03,연극/영화/극장,N03A06,...,135867,6093.0,NaN,103,NaN,NaN,127.044191,37.513142,1974.0,2021-01-19


In [63]:
data3 = data3[["상권업종대분류명","위도","경도"]]

In [64]:
data3

,상권업종대분류명,위도,경도
0,관광/여가/오락,37.528775,127.035280
1,관광/여가/오락,37.514850,127.019521
2,관광/여가/오락,37.485410,127.017045
3,관광/여가/오락,37.469075,127.040265
4,관광/여가/오락,37.477960,126.988447
...,...,...,...
1016,관광/여가/오락,37.511651,127.004451
1017,관광/여가/오락,37.522567,127.019997
1018,관광/여가/오락,37.512518,127.050613
1019,관광/여가/오락,37.513142,127.044191


In [65]:
data3['store_cnt']=1
data3['경도'] = data3['경도'].astype(float)
data3['위도'] = data3['위도'].astype(float)
data3['point'] = data3.apply(lambda row : Point([row['경도'], row['위도']]), axis=1)
data3 = gpd.GeoDataFrame(data3, geometry='point')

<ipython-input-65-9ea83613d36b>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data3['store_cnt']=1
<ipython-input-65-9ea83613d36b>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data3['경도'] = data3['경도'].astype(float)
<ipython-input-65-9ea83613d36b>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.ht

In [66]:
data3

,상권업종대분류명,위도,경도,store_cnt,point
0,관광/여가/오락,37.528775,127.035280,1,POINT (127.03528 37.52877)
1,관광/여가/오락,37.514850,127.019521,1,POINT (127.01952 37.51485)
2,관광/여가/오락,37.485410,127.017045,1,POINT (127.01705 37.48541)
3,관광/여가/오락,37.469075,127.040265,1,POINT (127.04026 37.46908)
4,관광/여가/오락,37.477960,126.988447,1,POINT (126.98845 37.47796)
...,...,...,...,...,...
1016,관광/여가/오락,37.511651,127.004451,1,POINT (127.00445 37.51165)
1017,관광/여가/오락,37.522567,127.019997,1,POINT (127.02000 37.52257)
1018,관광/여가/오락,37.512518,127.050613,1,POINT (127.05061 37.51252)
1019,관광/여가/오락,37.513142,127.044191,1,POINT (127.04419 37.51314)


In [67]:
gid_list = []
for i in tqdm(range(data3.shape[0])):
    point = data3['point'].iloc[i]
    if data_gid['geometry'].contains(point).sum() == 0:
        g1 = "Na"
    else :
        g = data_gid['gid'].loc[data_gid['geometry'].contains(point)]
        g1 = g.iloc[0]
    gid_list.append(g1)

100%|██████████████████████████████████████████████████████████████████████████████| 1021/1021 [02:31<00:00,  6.75it/s]


In [68]:
data3['gid'] = gid_list
data3['TL_cnt']=1
result3 = data3[['TL_cnt', 'gid']].groupby(['gid']).sum()
result3.reset_index(inplace=True)
data3 =result3


In [69]:
data3

,gid,TL_cnt
0,다사542420,1
1,다사542421,3
2,다사542422,2
3,다사542423,1
4,다사542424,1
...,...,...
633,다사648432,1
634,다사649416,1
635,다사649431,8
636,다사656419,1


In [70]:
data3.to_csv("output/서초강남관광여가오락전처리완료.csv")

## '학문/교육'

In [71]:
data3 = pd.read_csv("output/상업대분류/서초강남학문교육.csv")
data3

,Unnamed: 0,Unnamed: 0.1,상가업소번호,상호명,지점명,상권업종대분류코드,상권업종대분류명,상권업종중분류코드,상권업종중분류명,상권업종소분류코드,...,구우편번호,신우편번호,동정보,층정보,호정보,전화번호,경도,위도,상권번호,데이터기준일자
0,12,61,24523198,노바수학학원,NaN,R,학문/교육,R01,학원-보습교습입시,R01A01,...,135939,6335.0,NaN,NaN,606,02-2226-8286,127.075573,37.492649,NaN,2016-01-26
1,18,115,24046008,STA,NaN,R,학문/교육,R13,학문교육기타,R13A03,...,135506,6276.0,NaN,NaN,505,02-577-5325,127.049714,37.494201,NaN,2016-01-27
2,25,151,20966240,설악피아노학원,NaN,R,학문/교육,R05,학원-음악미술무용,R05A02,...,137723,6515.0,NaN,2,NaN,02-593-9292,127.015432,37.517844,NaN,2017-10-06
3,47,256,24575383,개념과유형수학교습소,NaN,R,학문/교육,R01,학원-보습교습입시,R01A01,...,135230,6350.0,NaN,4,NaN,02-6085-5262,127.079606,37.482850,NaN,2017-10-30
4,50,288,25493310,방배홍익미술,NaN,R,학문/교육,R05,학원-음악미술무용,R05A01,...,137060,6679.0,NaN,3,NaN,02-3487-3661,126.987525,37.482611,NaN,2017-10-11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3641,47667,315823,17191780,골드캐슬원룸텔,NaN,R,학문/교육,R10,도서관/독서실,R10A02,...,135859,6259.0,NaN,5,4,NaN,127.033531,37.489826,NaN,2020-12-07
3642,47669,315833,17230168,케이제이무용아카데미&재활운동센터,NaN,R,학문/교육,R05,학원-음악미술무용,R05A02,...,135953,6072.0,NaN,2,NaN,NaN,127.054754,37.522274,2034.0,2020-12-22
3643,47670,315834,17230115,뒤셀도르프바이올린슐레,NaN,R,학문/교육,R05,학원-음악미술무용,R05A02,...,135957,6013.0,101,NaN,1304,NaN,127.052389,37.526796,NaN,2020-12-22
3644,47680,315902,17436132,그림아트스튜디오,NaN,R,학문/교육,R05,학원-음악미술무용,R05A01,...,135949,6065.0,NaN,NaN,NaN,02-6082-0370,127.044823,37.521275,NaN,2021-01-14


In [72]:
data3 = data3[["상권업종대분류명","위도","경도"]]

In [73]:
data3

,상권업종대분류명,위도,경도
0,학문/교육,37.492649,127.075573
1,학문/교육,37.494201,127.049714
2,학문/교육,37.517844,127.015432
3,학문/교육,37.482850,127.079606
4,학문/교육,37.482611,126.987525
...,...,...,...
3641,학문/교육,37.489826,127.033531
3642,학문/교육,37.522274,127.054754
3643,학문/교육,37.526796,127.052389
3644,학문/교육,37.521275,127.044823


In [74]:
data3['store_cnt']=1
data3['경도'] = data3['경도'].astype(float)
data3['위도'] = data3['위도'].astype(float)
data3['point'] = data3.apply(lambda row : Point([row['경도'], row['위도']]), axis=1)
data3 = gpd.GeoDataFrame(data3, geometry='point')

<ipython-input-74-9ea83613d36b>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data3['store_cnt']=1
<ipython-input-74-9ea83613d36b>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data3['경도'] = data3['경도'].astype(float)
<ipython-input-74-9ea83613d36b>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.ht

In [75]:
data3

,상권업종대분류명,위도,경도,store_cnt,point
0,학문/교육,37.492649,127.075573,1,POINT (127.07557 37.49265)
1,학문/교육,37.494201,127.049714,1,POINT (127.04971 37.49420)
2,학문/교육,37.517844,127.015432,1,POINT (127.01543 37.51784)
3,학문/교육,37.482850,127.079606,1,POINT (127.07961 37.48285)
4,학문/교육,37.482611,126.987525,1,POINT (126.98753 37.48261)
...,...,...,...,...,...
3641,학문/교육,37.489826,127.033531,1,POINT (127.03353 37.48983)
3642,학문/교육,37.522274,127.054754,1,POINT (127.05475 37.52227)
3643,학문/교육,37.526796,127.052389,1,POINT (127.05239 37.52680)
3644,학문/교육,37.521275,127.044823,1,POINT (127.04482 37.52127)


In [76]:
gid_list = []
for i in tqdm(range(data3.shape[0])):
    point = data3['point'].iloc[i]
    if data_gid['geometry'].contains(point).sum() == 0:
        g1 = "Na"
    else :
        g = data_gid['gid'].loc[data_gid['geometry'].contains(point)]
        g1 = g.iloc[0]
    gid_list.append(g1)

100%|██████████████████████████████████████████████████████████████████████████████| 3646/3646 [08:36<00:00,  7.05it/s]


In [77]:
data3['gid'] = gid_list
data3['TL_cnt']=1
result3 = data3[['TL_cnt', 'gid']].groupby(['gid']).sum()
result3.reset_index(inplace=True)
data3 =result3

In [78]:
data3

,gid,TL_cnt
0,다사542417,1
1,다사542418,2
2,다사542421,1
3,다사542422,1
4,다사542423,1
...,...,...
1374,다사655418,1
1375,다사656419,3
1376,다사657418,1
1377,다사658419,2


In [79]:
data3.to_csv("output/서초강남학문교육전처리완료.csv")

##  '숙박'

In [80]:
data3 = pd.read_csv("output/상업대분류/서초강남숙박.csv")
data3

,Unnamed: 0,Unnamed: 0.1,상가업소번호,상호명,지점명,상권업종대분류코드,상권업종대분류명,상권업종중분류코드,상권업종중분류명,상권업종소분류코드,...,구우편번호,신우편번호,동정보,층정보,호정보,전화번호,경도,위도,상권번호,데이터기준일자
0,29,161,28494447,아트호텔,NaN,O,숙박,O01,호텔/콘도,O01A01,...,137070,6652.0,NaN,1.0,NaN,02-584-6701,127.012875,37.486952,1845.0,2017-09-23
1,66,396,15655467,블루펄호텔,NaN,O,숙박,O01,호텔/콘도,O01A01,...,135812,6040.0,NaN,NaN,NaN,NaN,127.025433,37.517809,NaN,2017-10-02
2,236,1488,15496894,강남스테이호텔,NaN,O,숙박,O01,호텔/콘도,O01A01,...,135924,6236.0,S,NaN,1.0,NaN,127.035443,37.499411,NaN,2017-09-30
3,638,3656,28510235,알로프트호텔서울강남(2014년10월개관예정),NaN,O,숙박,O01,호텔/콘도,O01A01,...,135957,6075.0,NaN,1.0,NaN,02-510-9700,127.055641,37.523731,NaN,2017-08-26
4,759,4366,28507200,인더시티레지던스,NaN,O,숙박,O01,호텔/콘도,O01A01,...,135010,6110.0,1,1.0,NaN,NaN,127.023502,37.510531,1967.0,2017-09-17
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
227,45570,302723,23317162,화이트모텔,NaN,O,숙박,O02,모텔/여관/여인숙,O02A01,...,135080,6141.0,NaN,1.0,NaN,02-564-5286,127.038489,37.502616,1928.0,2017-09-25
228,45840,304685,17242833,잠원동렌탈하우스,NaN,O,숙박,O03,캠프/별장/펜션,O03A01,...,137030,6518.0,101,NaN,1202.0,NaN,127.011702,37.515046,NaN,2020-12-23
229,46560,308851,23284541,강남아르누보씨티호텔,NaN,O,숙박,O01,호텔/콘도,O01A01,...,137070,6626.0,NaN,3.0,NaN,02-580-7500,127.028125,37.493404,1887.0,2017-10-03
230,47024,311777,23073694,유원메이트관광호텔,NaN,O,숙박,O01,호텔/콘도,O01A01,...,137070,6653.0,NaN,2.0,NaN,02-585-0561,127.012818,37.485240,1845.0,2017-09-23


In [81]:
data3 = data3[["상권업종대분류명","위도","경도"]]

In [82]:
data3

,상권업종대분류명,위도,경도
0,숙박,37.486952,127.012875
1,숙박,37.517809,127.025433
2,숙박,37.499411,127.035443
3,숙박,37.523731,127.055641
4,숙박,37.510531,127.023502
...,...,...,...
227,숙박,37.502616,127.038489
228,숙박,37.515046,127.011702
229,숙박,37.493404,127.028125
230,숙박,37.485240,127.012818


In [83]:
data3['store_cnt']=1
data3['경도'] = data3['경도'].astype(float)
data3['위도'] = data3['위도'].astype(float)
data3['point'] = data3.apply(lambda row : Point([row['경도'], row['위도']]), axis=1)
data3 = gpd.GeoDataFrame(data3, geometry='point')

<ipython-input-83-9ea83613d36b>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data3['store_cnt']=1
<ipython-input-83-9ea83613d36b>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data3['경도'] = data3['경도'].astype(float)
<ipython-input-83-9ea83613d36b>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.ht

In [84]:
data3

,상권업종대분류명,위도,경도,store_cnt,point
0,숙박,37.486952,127.012875,1,POINT (127.01288 37.48695)
1,숙박,37.517809,127.025433,1,POINT (127.02543 37.51781)
2,숙박,37.499411,127.035443,1,POINT (127.03544 37.49941)
3,숙박,37.523731,127.055641,1,POINT (127.05564 37.52373)
4,숙박,37.510531,127.023502,1,POINT (127.02350 37.51053)
...,...,...,...,...,...
227,숙박,37.502616,127.038489,1,POINT (127.03849 37.50262)
228,숙박,37.515046,127.011702,1,POINT (127.01170 37.51505)
229,숙박,37.493404,127.028125,1,POINT (127.02812 37.49340)
230,숙박,37.485240,127.012818,1,POINT (127.01282 37.48524)


In [85]:
gid_list = []
for i in tqdm(range(data3.shape[0])):
    point = data3['point'].iloc[i]
    if data_gid['geometry'].contains(point).sum() == 0:
        g1 = "Na"
    else :
        g = data_gid['gid'].loc[data_gid['geometry'].contains(point)]
        g1 = g.iloc[0]
    gid_list.append(g1)

100%|████████████████████████████████████████████████████████████████████████████████| 232/232 [00:32<00:00,  7.15it/s]


In [86]:
data3['gid'] = gid_list
data3['TL_cnt']=1
result3 = data3[['TL_cnt', 'gid']].groupby(['gid']).sum()
result3.reset_index(inplace=True)
data3 =result3


In [87]:
data3

,gid,TL_cnt
0,다사542433,1
1,다사543427,2
2,다사545426,1
3,다사545440,2
4,다사545443,1
...,...,...
160,다사613457,3
161,다사614455,1
162,다사615455,1
163,다사616456,1


In [88]:
data3.to_csv("output/서초강남숙박전처리완료.csv")

##  '스포츠'

In [89]:
data3 = pd.read_csv("output/상업대분류/서초강남스포츠.csv")
data3

,Unnamed: 0,Unnamed: 0.1,상가업소번호,상호명,지점명,상권업종대분류코드,상권업종대분류명,상권업종중분류코드,상권업종중분류명,상권업종소분류코드,...,구우편번호,신우편번호,동정보,층정보,호정보,전화번호,경도,위도,상권번호,데이터기준일자
0,1482,8767,16173497,블록체인과암호화폐,NaN,P,스포츠,P01,운영관리시설,P01A99,...,135509,6153.0,NaN,9,901,NaN,127.049234,37.511543,NaN,2018-08-29
1,2027,12125,16423558,KKM,NaN,P,스포츠,P02,실내운동시설,P02A01,...,135820,6059.0,NaN,3,NaN,NaN,127.040065,37.518254,1991.0,2020-03-02
2,2102,12597,16423452,커넥스터,NaN,P,스포츠,P02,실내운동시설,P02A01,...,137898,6784.0,NaN,5,NaN,02-573-1333,127.045747,37.471796,NaN,2020-03-02
3,2104,12604,16421920,루카스포츠,NaN,P,스포츠,P02,실내운동시설,P02A01,...,135820,6059.0,NaN,5,502,NaN,127.040261,37.517967,1991.0,2020-03-02
4,2239,13422,16202500,짱뽕킹,FIT,P,스포츠,P02,실내운동시설,P02A13,...,135893,6025.0,NaN,NaN,1,02-543-0806,127.028506,37.521583,2053.0,2018-09-14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
147,46449,308285,17182437,플레이존스포츠,NaN,P,스포츠,P02,실내운동시설,P02A14,...,135888,6028.0,NaN,1,NaN,NaN,127.020323,37.522696,2045.0,2020-12-07
148,46683,309647,17186013,아이러브스포츠,NaN,P,스포츠,P02,실내운동시설,P02A14,...,137877,6643.0,NaN,NaN,102,NaN,127.020364,37.486708,NaN,2020-12-07
149,47370,313767,17452770,스포츠재활재활PT운동진정성,NaN,P,스포츠,P02,실내운동시설,P02A01,...,137840,6588.0,NaN,NaN,201,NaN,126.996066,37.489919,NaN,2021-02-04
150,47472,314386,17452889,스튜디오베이스,NaN,P,스포츠,P02,실내운동시설,P02A01,...,135555,6295.0,NaN,22,14,NaN,127.051842,37.486911,NaN,2021-02-04


In [90]:
data3 = data3[["상권업종대분류명","위도","경도"]]

In [91]:
data3

,상권업종대분류명,위도,경도
0,스포츠,37.511543,127.049234
1,스포츠,37.518254,127.040065
2,스포츠,37.471796,127.045747
3,스포츠,37.517967,127.040261
4,스포츠,37.521583,127.028506
...,...,...,...
147,스포츠,37.522696,127.020323
148,스포츠,37.486708,127.020364
149,스포츠,37.489919,126.996066
150,스포츠,37.486911,127.051842


In [92]:
data3['store_cnt']=1
data3['경도'] = data3['경도'].astype(float)
data3['위도'] = data3['위도'].astype(float)
data3['point'] = data3.apply(lambda row : Point([row['경도'], row['위도']]), axis=1)
data3 = gpd.GeoDataFrame(data3, geometry='point')

<ipython-input-92-9ea83613d36b>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data3['store_cnt']=1
<ipython-input-92-9ea83613d36b>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data3['경도'] = data3['경도'].astype(float)
<ipython-input-92-9ea83613d36b>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.ht

In [93]:
data3

,상권업종대분류명,위도,경도,store_cnt,point
0,스포츠,37.511543,127.049234,1,POINT (127.04923 37.51154)
1,스포츠,37.518254,127.040065,1,POINT (127.04006 37.51825)
2,스포츠,37.471796,127.045747,1,POINT (127.04575 37.47180)
3,스포츠,37.517967,127.040261,1,POINT (127.04026 37.51797)
4,스포츠,37.521583,127.028506,1,POINT (127.02851 37.52158)
...,...,...,...,...,...
147,스포츠,37.522696,127.020323,1,POINT (127.02032 37.52270)
148,스포츠,37.486708,127.020364,1,POINT (127.02036 37.48671)
149,스포츠,37.489919,126.996066,1,POINT (126.99607 37.48992)
150,스포츠,37.486911,127.051842,1,POINT (127.05184 37.48691)


In [94]:
gid_list = []
for i in tqdm(range(data3.shape[0])):
    point = data3['point'].iloc[i]
    if data_gid['geometry'].contains(point).sum() == 0:
        g1 = "Na"
    else :
        g = data_gid['gid'].loc[data_gid['geometry'].contains(point)]
        g1 = g.iloc[0]
    gid_list.append(g1)

100%|████████████████████████████████████████████████████████████████████████████████| 152/152 [00:20<00:00,  7.49it/s]


In [95]:
data3['gid'] = gid_list
data3['TL_cnt']=1
result3 = data3[['TL_cnt', 'gid']].groupby(['gid']).sum()
result3.reset_index(inplace=True)
data3 =result3


In [96]:
data3

,gid,TL_cnt
0,다사542418,1
1,다사543426,1
2,다사543429,1
3,다사543443,1
4,다사545437,1
...,...,...
136,다사621444,1
137,다사628439,2
138,다사632437,1
139,다사633438,1


In [97]:
data3.to_csv("output/서초강남스포츠전처리완료.csv")

# 과속방지턱 전처리

In [3]:
data3 = pd.read_csv("output/서초강남과속방지턱.csv")
data3

,Unnamed: 0,과속방지턱관리번호,시도명,시군구명,도로명,소재지도로명주소,소재지지번주소,설치장소,과속방지턱재료,과속방지턱형태구분,...,위도,경도,보차분리여부,연속형여부,과속방지턱설치년도,관리기관명,관리기관전화번호,데이터기준일자,제공기관코드,제공기관명
0,1391,21.0,01,서초구,경부고속도로,NaN,서울특별시 서초구 서초동 1301-11,경부고속도로(서울방향) 반포IC 진출램프,99,3,...,37.501648,127.019357,N,N,2006.0,서울시설공단,02-2290-6363,2019-06-19,6110000.0,서울특별시
1,1392,20.0,01,서초구,경부고속도로,NaN,서울특별시 서초구 잠원동 86,경부고속도로(부산방향) 잠원IC 진출램프,99,3,...,37.509701,127.015514,N,Y,NaN,서울시설공단,02-2290-6363,2019-06-19,6110000.0,서울특별시
2,1393,19.0,01,서초구,경부고속도로,NaN,서울특별시 서초구 잠원동 86,경부고속도로(부산방향) 잠원IC 진출램프,99,3,...,37.509701,127.015514,N,Y,2006.0,서울시설공단,02-2290-6363,2019-06-19,6110000.0,서울특별시
3,5060,73.0,01,강남구,헌릉로,NaN,서울특별시 강남구 세곡동 115-3,대왕파출소 앞,99,3,...,37.465301,127.108053,Y,N,2015.0,서울특별시 동부도로사업소,02-2040-0344,2019-06-19,6110000.0,서울특별시
4,12136,1.0,서울특별시,강남구,광평로20길,NaN,서울특별시 강남구 일원동 734-2,대왕중학교 앞,1,1,...,37.483241,127.086698,Y,Y,NaN,서울특별시 강남구청,02-3423-6565,2020-09-01,3220000.0,서울특별시 강남구
5,12137,2.0,서울특별시,강남구,광평로20길,NaN,서울특별시 강남구 일원동 734-2,대왕중학교 앞,1,1,...,37.483123,127.086745,Y,Y,NaN,서울특별시 강남구청,02-3423-6565,2020-09-01,3220000.0,서울특별시 강남구
6,12138,3.0,서울특별시,강남구,광평로20길,NaN,서울특별시 강남구 일원동 734-2,남서울은혜교회 앞,1,1,...,37.482274,127.086876,Y,Y,NaN,서울특별시 강남구청,02-3423-6565,2020-09-01,3220000.0,서울특별시 강남구
7,12139,4.0,서울특별시,강남구,광평로20길,NaN,서울특별시 강남구 일원동 734-2,남서울은혜교회 앞,1,1,...,37.481931,127.086736,Y,Y,NaN,서울특별시 강남구청,02-3423-6565,2020-09-01,3220000.0,서울특별시 강남구
8,12140,5.0,서울특별시,강남구,광평로20길,NaN,서울특별시 강남구 일원동 734-2,서울대모초등학교 앞,1,1,...,37.481689,127.086442,Y,Y,NaN,서울특별시 강남구청,02-3423-6565,2020-09-01,3220000.0,서울특별시 강남구
9,12141,6.0,서울특별시,강남구,광평로20길,NaN,서울특별시 강남구 일원동 734-3,서울로봇고등학교 앞,1,1,...,37.481477,127.085426,Y,Y,NaN,서울특별시 강남구청,02-3423-6565,2020-09-01,3220000.0,서울특별시 강남구


In [4]:
data3 = data3[["도로명","위도","경도"]]

In [5]:
data3

,도로명,위도,경도
0,경부고속도로,37.501648,127.019357
1,경부고속도로,37.509701,127.015514
2,경부고속도로,37.509701,127.015514
3,헌릉로,37.465301,127.108053
4,광평로20길,37.483241,127.086698
5,광평로20길,37.483123,127.086745
6,광평로20길,37.482274,127.086876
7,광평로20길,37.481931,127.086736
8,광평로20길,37.481689,127.086442
9,광평로20길,37.481477,127.085426


In [6]:
data3['sub_cnt']=1
data3['경도'] = data3['경도'].astype(float)
data3['위도'] = data3['위도'].astype(float)
data3['point'] = data3.apply(lambda row : Point([row['경도'], row['위도']]), axis=1)
data3 = gpd.GeoDataFrame(data3, geometry='point')

<ipython-input-6-9f078b38371f>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data3['sub_cnt']=1
<ipython-input-6-9f078b38371f>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data3['경도'] = data3['경도'].astype(float)
<ipython-input-6-9f078b38371f>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#re

In [7]:
data3

,도로명,위도,경도,sub_cnt,point
0,경부고속도로,37.501648,127.019357,1,POINT (127.01936 37.50165)
1,경부고속도로,37.509701,127.015514,1,POINT (127.01551 37.50970)
2,경부고속도로,37.509701,127.015514,1,POINT (127.01551 37.50970)
3,헌릉로,37.465301,127.108053,1,POINT (127.10805 37.46530)
4,광평로20길,37.483241,127.086698,1,POINT (127.08670 37.48324)
5,광평로20길,37.483123,127.086745,1,POINT (127.08675 37.48312)
6,광평로20길,37.482274,127.086876,1,POINT (127.08688 37.48227)
7,광평로20길,37.481931,127.086736,1,POINT (127.08674 37.48193)
8,광평로20길,37.481689,127.086442,1,POINT (127.08644 37.48169)
9,광평로20길,37.481477,127.085426,1,POINT (127.08543 37.48148)


In [13]:
gid_list = []
for i in tqdm(range(data3.shape[0])):
    point = data3['point'].iloc[i]
    if data_gid['geometry'].contains(point).sum() == 0:
        g1 = "Na"
    else :
        g = data_gid['gid'].loc[data_gid['geometry'].contains(point)]
        g1 = g.iloc[0]
    gid_list.append(g1)

100%|██████████████████████████████████████████████████████████████████████████████████| 42/42 [00:05<00:00,  8.23it/s]


In [14]:
data3['gid'] = gid_list
data3['TL_cnt']=1
result3 = data3[['TL_cnt', 'gid']].groupby(['gid']).sum()
result3.reset_index(inplace=True)
data3 =result3


In [15]:
data3

,gid,TL_cnt
0,다사571457,2
1,다사575448,1
2,다사575455,1
3,다사576454,1
4,다사615393,1
5,다사615394,2
6,다사615395,2
7,다사616392,2
8,다사616396,2
9,다사617396,2


In [16]:
data3.to_csv("output/서초강남과속방지턱전처리완료.csv")

# 서초 강남 사고 전처리

In [17]:
data3 = pd.read_csv("output/서초강남PM사고위치.csv")
data3

,Unnamed: 0,사고번호,사고일시,요일,위도,경도,시군구,사고내용,사망자수,중상자수,...,기상상태,도로형태,가해운전자 차종,가해운전자 성별,가해운전자 연령,가해운전자 상해정도,피해운전자 차종,피해운전자 성별,피해운전자 연령,피해운전자 상해정도
0,0,2017032400100162,2017년 3월 24일 09시,금요일,37.482441794881645,127.03594277561041,서울특별시 서초구 양재동,경상사고,0,0,...,맑음,기타 - 기타,개인형이동수단(PM),기타불명,미분류,기타불명,보행자,여,28세,경상
1,2,2017042300100461,2017년 4월 23일 20시,일요일,37.47409693517618,126.992761936558,서울특별시 서초구 방배동,경상사고,0,0,...,맑음,단일로 - 기타,개인형이동수단(PM),남,74세,경상,승용,여,55세,상해없음
2,5,2017052500100328,2017년 5월 25일 14시,목요일,37.493402432997534,127.01601773561951,서울특별시 서초구 서초동,중상사고,0,1,...,맑음,단일로 - 기타,개인형이동수단(PM),남,20세,상해없음,보행자,여,63세,중상
3,9,2017062800100470,2017년 6월 28일 19시,수요일,37.51066450830801,127.069254621366,서울특별시 강남구 논현동,중상사고,0,1,...,맑음,단일로 - 기타,개인형이동수단(PM),남,20세,상해없음,자전거,남,61세,중상
4,37,2018042000100032,2018년 4월 20일 02시,금요일,37.50027533962582,127.0621995169631,서울특별시 강남구 대치동,부상신고사고,0,0,...,흐림,기타 - 기타,개인형이동수단(PM),남,35세,상해없음,보행자,남,39세,부상신고
5,63,2018091700100054,2018년 9월 17일 06시,월요일,37.48947164056322,126.98292869107078,서울특별시 서초구 방배동,경상사고,0,0,...,맑음,교차로 - 교차로안,개인형이동수단(PM),남,28세,경상,화물,남,54세,상해없음
6,72,2018113000100116,2018년 11월 30일 08시,금요일,37.4878426967116,127.10243126911546,서울특별시 강남구 수서동,중상사고,0,1,...,맑음,단일로 - 기타,개인형이동수단(PM),여,23세,상해없음,보행자,남,33세,중상
7,76,2018121100100493,2018년 12월 11일 19시,화요일,37.52282209699904,127.01872203181729,서울특별시 강남구 압구정동,중상사고,0,1,...,맑음,단일로 - 기타,개인형이동수단(PM),남,28세,중상,승용,여,42세,상해없음
8,83,2019021200100506,2019년 2월 12일 22시,화요일,37.50641397668101,127.02342851902121,서울특별시 강남구 논현동,중상사고,0,2,...,맑음,단일로 - 기타,개인형이동수단(PM),남,29세,중상,이륜,남,31세,중상
9,84,2019031300100215,2019년 3월 13일 11시,수요일,37.48918508906506,127.02012987685616,서울특별시 서초구 서초동,중상사고,0,1,...,맑음,단일로 - 기타,개인형이동수단(PM),남,22세,경상,이륜,남,26세,중상


In [18]:
data3 = data3[["사고내용","위도","경도"]]

In [19]:
data3

,사고내용,위도,경도
0,경상사고,37.482441794881645,127.03594277561041
1,경상사고,37.47409693517618,126.992761936558
2,중상사고,37.493402432997534,127.01601773561951
3,중상사고,37.51066450830801,127.069254621366
4,부상신고사고,37.50027533962582,127.0621995169631
5,경상사고,37.48947164056322,126.98292869107078
6,중상사고,37.4878426967116,127.10243126911546
7,중상사고,37.52282209699904,127.01872203181729
8,중상사고,37.50641397668101,127.02342851902121
9,중상사고,37.48918508906506,127.02012987685616


In [20]:
data3['sub_cnt']=1
data3['경도'] = data3['경도'].astype(float)
data3['위도'] = data3['위도'].astype(float)
data3['point'] = data3.apply(lambda row : Point([row['경도'], row['위도']]), axis=1)
data3 = gpd.GeoDataFrame(data3, geometry='point')

<ipython-input-20-9f078b38371f>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data3['sub_cnt']=1
<ipython-input-20-9f078b38371f>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data3['경도'] = data3['경도'].astype(float)
<ipython-input-20-9f078b38371f>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html

In [21]:
data3

,사고내용,위도,경도,sub_cnt,point
0,경상사고,37.482442,127.035943,1,POINT (127.03594 37.48244)
1,경상사고,37.474097,126.992762,1,POINT (126.99276 37.47410)
2,중상사고,37.493402,127.016018,1,POINT (127.01602 37.49340)
3,중상사고,37.510665,127.069255,1,POINT (127.06925 37.51066)
4,부상신고사고,37.500275,127.062200,1,POINT (127.06220 37.50028)
5,경상사고,37.489472,126.982929,1,POINT (126.98293 37.48947)
6,중상사고,37.487843,127.102431,1,POINT (127.10243 37.48784)
7,중상사고,37.522822,127.018722,1,POINT (127.01872 37.52282)
8,중상사고,37.506414,127.023429,1,POINT (127.02343 37.50641)
9,중상사고,37.489185,127.020130,1,POINT (127.02013 37.48919)


In [22]:
gid_list = []
for i in tqdm(range(data3.shape[0])):
    point = data3['point'].iloc[i]
    if data_gid['geometry'].contains(point).sum() == 0:
        g1 = "Na"
    else :
        g = data_gid['gid'].loc[data_gid['geometry'].contains(point)]
        g1 = g.iloc[0]
    gid_list.append(g1)

100%|██████████████████████████████████████████████████████████████████████████████████| 60/60 [00:08<00:00,  7.45it/s]


In [23]:
data3['gid'] = gid_list
data3['TL_cnt']=1
result3 = data3[['TL_cnt', 'gid']].groupby(['gid']).sum()
result3.reset_index(inplace=True)
data3 =result3


In [24]:
data3

,gid,TL_cnt
0,다사542434,1
1,다사547430,1
2,다사549419,1
3,다사551417,1
4,다사564435,1
5,다사566432,1
6,다사567450,1
7,다사568465,1
8,다사569441,1
9,다사570474,1


In [25]:
data3.to_csv("output/서초강남PM교통사고전처리완료.csv")

# 학교 전처리

In [29]:
data3 = pd.read_csv("output/서초구강남구초중고대학교.csv")
data3

,Unnamed: 0,학교명,소재지도로명주소,위도,경도
0,0,가톨릭대학교,서울특별시 서초구 반포대로 222,37.502394,127.005861
1,1,한국예술종합학교,서울특별시 서초구 남부순환로 2374,37.477919,127.010770
2,2,서울외국어대학원대학교,서울특별시 서초구 남부순환로356길 85,37.481279,127.038956
3,3,예명대학원대학교,서울특별시 서초구 남부순환로337가길 49,37.484222,127.023745
4,4,치유상담대학원대학교,서울특별시 서초구 서초대로 121,37.488530,126.995461
...,...,...,...,...,...
133,133,신사중학교,서울특별시 강남구 압구정로 123,37.524329,127.021236
134,134,서울일원초등학교,서울특별시 강남구 영동대로4길 20,37.489473,127.076367
135,135,내곡중학교,서울특별시 서초구 청계산로7길 9-20,37.451982,127.054783
136,136,서울양재초등학교,서울특별시 서초구 바우뫼로18길 14,37.473810,127.031886


In [30]:
data3 = data3[["학교명","위도","경도"]]

In [31]:
data3

,학교명,위도,경도
0,가톨릭대학교,37.502394,127.005861
1,한국예술종합학교,37.477919,127.010770
2,서울외국어대학원대학교,37.481279,127.038956
3,예명대학원대학교,37.484222,127.023745
4,치유상담대학원대학교,37.488530,126.995461
...,...,...,...
133,신사중학교,37.524329,127.021236
134,서울일원초등학교,37.489473,127.076367
135,내곡중학교,37.451982,127.054783
136,서울양재초등학교,37.473810,127.031886


In [32]:
data3['sub_cnt']=1
data3['경도'] = data3['경도'].astype(float)
data3['위도'] = data3['위도'].astype(float)
data3['point'] = data3.apply(lambda row : Point([row['경도'], row['위도']]), axis=1)
data3 = gpd.GeoDataFrame(data3, geometry='point')

<ipython-input-32-9f078b38371f>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data3['sub_cnt']=1
<ipython-input-32-9f078b38371f>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data3['경도'] = data3['경도'].astype(float)
<ipython-input-32-9f078b38371f>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html

In [33]:
data3

,학교명,위도,경도,sub_cnt,point
0,가톨릭대학교,37.502394,127.005861,1,POINT (127.00586 37.50239)
1,한국예술종합학교,37.477919,127.010770,1,POINT (127.01077 37.47792)
2,서울외국어대학원대학교,37.481279,127.038956,1,POINT (127.03896 37.48128)
3,예명대학원대학교,37.484222,127.023745,1,POINT (127.02374 37.48422)
4,치유상담대학원대학교,37.488530,126.995461,1,POINT (126.99546 37.48853)
...,...,...,...,...,...
133,신사중학교,37.524329,127.021236,1,POINT (127.02124 37.52433)
134,서울일원초등학교,37.489473,127.076367,1,POINT (127.07637 37.48947)
135,내곡중학교,37.451982,127.054783,1,POINT (127.05478 37.45198)
136,서울양재초등학교,37.473810,127.031886,1,POINT (127.03189 37.47381)


In [34]:
gid_list = []
for i in tqdm(range(data3.shape[0])):
    point = data3['point'].iloc[i]
    if data_gid['geometry'].contains(point).sum() == 0:
        g1 = "Na"
    else :
        g = data_gid['gid'].loc[data_gid['geometry'].contains(point)]
        g1 = g.iloc[0]
    gid_list.append(g1)

100%|████████████████████████████████████████████████████████████████████████████████| 138/138 [00:16<00:00,  8.36it/s]


In [35]:
data3['gid'] = gid_list
data3['TL_cnt']=1
result3 = data3[['TL_cnt', 'gid']].groupby(['gid']).sum()
result3.reset_index(inplace=True)
data3 =result3


In [36]:
data3

,gid,TL_cnt
0,다사544422,1
1,다사544434,1
2,다사545434,1
3,다사546432,1
4,다사547412,1
...,...,...
129,다사647436,1
130,다사648434,1
131,다사648435,1
132,다사651406,1


In [37]:
data3.to_csv("output/그리드 전처리 완료/서초강남학교전처리완료.csv")

## cctv

In [58]:
data3 = pd.read_csv("output/서초강남cctv.csv")
data3

,Unnamed: 0,Unnamed: 0.1,고정형CCTV지번주소,위도,경도,자치구,단속지점명,현장구분
0,50,50,방배본동 19-3,37.495498,126.988840,서초구,서래초병설유치원주변,불법주정차구역
1,51,51,방배1동 906-13,37.483566,126.997070,서초구,서리풀e편한세상APT 주변,불법주정차구역
2,52,52,방배1동 906-16,37.484531,126.997412,서초구,서리풀e편한세상APT2번출구주변,불법주정차구역
3,53,53,반포4동 51-5,37.501277,127.011311,서초구,서석빌딩사거리 주변,불법주정차구역
4,54,54,서초2동 1325-1,37.493085,127.024155,서초구,서운중학교뒷길 주변,불법주정차구역
...,...,...,...,...,...,...,...,...
674,3311,3311,서울 서초구 잠원동 62-4,37.513444,127.010149,서초구,신반포한신347동,불법주정차구역
675,3314,3314,서울 강남구 삼성동 146-28,37.512300,127.055749,강남구,[삼성1-125] 태형빌딩 주변,불법주정차구역
676,3315,3315,서울 강남구 삼성동 29-19,37.514338,127.045811,강남구,[삼성2-119] 삼성벨라스아파트 주변,불법주정차구역
677,3319,3319,서울 강남구 삼성동 141-19,37.505168,127.048947,강남구,[삼성2-113] 선릉역8번출구 주변,불법주정차구역


In [59]:
data3 = data3[["단속지점명","위도","경도"]]

In [60]:
data3

,단속지점명,위도,경도
0,서래초병설유치원주변,37.495498,126.988840
1,서리풀e편한세상APT 주변,37.483566,126.997070
2,서리풀e편한세상APT2번출구주변,37.484531,126.997412
3,서석빌딩사거리 주변,37.501277,127.011311
4,서운중학교뒷길 주변,37.493085,127.024155
...,...,...,...
674,신반포한신347동,37.513444,127.010149
675,[삼성1-125] 태형빌딩 주변,37.512300,127.055749
676,[삼성2-119] 삼성벨라스아파트 주변,37.514338,127.045811
677,[삼성2-113] 선릉역8번출구 주변,37.505168,127.048947


In [61]:
data3['sub_cnt']=1
data3['경도'] = data3['경도'].astype(float)
data3['위도'] = data3['위도'].astype(float)
data3['point'] = data3.apply(lambda row : Point([row['경도'], row['위도']]), axis=1)
data3 = gpd.GeoDataFrame(data3, geometry='point')

<ipython-input-61-9f078b38371f>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data3['sub_cnt']=1
<ipython-input-61-9f078b38371f>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data3['경도'] = data3['경도'].astype(float)
<ipython-input-61-9f078b38371f>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html

In [62]:
data3

,단속지점명,위도,경도,sub_cnt,point
0,서래초병설유치원주변,37.495498,126.988840,1,POINT (126.98884 37.49550)
1,서리풀e편한세상APT 주변,37.483566,126.997070,1,POINT (126.99707 37.48357)
2,서리풀e편한세상APT2번출구주변,37.484531,126.997412,1,POINT (126.99741 37.48453)
3,서석빌딩사거리 주변,37.501277,127.011311,1,POINT (127.01131 37.50128)
4,서운중학교뒷길 주변,37.493085,127.024155,1,POINT (127.02415 37.49309)
...,...,...,...,...,...
674,신반포한신347동,37.513444,127.010149,1,POINT (127.01015 37.51344)
675,[삼성1-125] 태형빌딩 주변,37.512300,127.055749,1,POINT (127.05575 37.51230)
676,[삼성2-119] 삼성벨라스아파트 주변,37.514338,127.045811,1,POINT (127.04581 37.51434)
677,[삼성2-113] 선릉역8번출구 주변,37.505168,127.048947,1,POINT (127.04895 37.50517)


In [63]:
gid_list = []
for i in tqdm(range(data3.shape[0])):
    point = data3['point'].iloc[i]
    if data_gid['geometry'].contains(point).sum() == 0:
        g1 = "Na"
    else :
        g = data_gid['gid'].loc[data_gid['geometry'].contains(point)]
        g1 = g.iloc[0]
    gid_list.append(g1)

100%|████████████████████████████████████████████████████████████████████████████████| 679/679 [01:39<00:00,  6.81it/s]


In [64]:
data3['gid'] = gid_list
data3['TL_cnt']=1
result3 = data3[['TL_cnt', 'gid']].groupby(['gid']).sum()
result3.reset_index(inplace=True)
data3 =result3


In [65]:
data3

,gid,TL_cnt
0,다사541418,1
1,다사541421,1
2,다사542421,1
3,다사542422,1
4,다사542429,1
...,...,...
638,다사662413,1
639,다사663411,1
640,다사663412,1
641,다사664410,1


In [66]:
data3.to_csv("output/서초강남cctv전처리완료.csv")

## 무인단속

In [49]:
data3 = pd.read_csv("output/서초강남무인단속.csv")
data3

,Unnamed: 0,Unnamed: 0.1,무인교통단속카메라관리번호,시도명,시군구명,도로종류,도로노선번호,도로노선명,도로노선방향,소재지도로명주소,...,제한속도,단속구간위치구분,과속단속구간길이,보호구역구분,설치연도,관리기관명,관리기관전화번호,데이터기준일자,제공기관코드,제공기관명
0,141,1269,F9956,서울특별시,서초구,특별시도,NaN,남부순환로,2.0,서울특별시 서초구 남부순환로 2183,...,60,NaN,NaN,99.0,2017.0,서울지방경찰청,02-700-5031,2020-05-31,1320000,경찰청
1,142,1270,F9957,서울특별시,서초구,일반국도,47번,중앙로,2.0,서울특별시 서초구 중앙로,...,60,NaN,NaN,99.0,2017.0,서울지방경찰청,02-700-5031,2020-05-31,1320000,경찰청
2,147,1287,F9342,서울특별시,서초구,특별시도,NaN,효령로,1.0,서울특별시 서초구 효령로,...,50,NaN,NaN,99.0,2016.0,서울지방경찰청,02-700-5031,2020-05-31,1320000,경찰청
3,157,1297,F9352,서울특별시,강남구,특별시도,NaN,남부순환로,2.0,서울특별시 강남구 남부순환로 2726,...,60,NaN,NaN,99.0,2016.0,서울지방경찰청,02-700-5031,2020-05-31,1320000,경찰청
4,165,1305,F9360,서울특별시,서초구,특별시도,NaN,잠원로,1.0,서울특별시 서초구 잠원로,...,60,NaN,NaN,99.0,2016.0,서울지방경찰청,02-700-5031,2020-05-31,1320000,경찰청
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91,1224,8386,F4729,서울특별시,강남구,지방도,23번,양재대로,1.0,서울특별시 강남구 양재대로,...,70,NaN,NaN,99.0,2009.0,서울지방경찰청,02-700-5031,2020-05-31,1320000,경찰청
92,1229,8391,F4736,서울특별시,강남구,특별시도,NaN,도산대로,2.0,서울특별시 강남구 도산대로 502,...,60,NaN,NaN,99.0,2009.0,서울지방경찰청,02-700-5031,2020-05-31,1320000,경찰청
93,1240,8402,F5035,서울특별시,강남구,특별시도,NaN,개포로,1.0,서울특별시 강남구 개포로,...,40,NaN,NaN,99.0,2009.0,서울지방경찰청,02-700-5031,2020-05-31,1320000,경찰청
94,1262,8424,F4416,서울특별시,강남구,지방도,23번,남부순환로,2.0,서울특별시 강남구 남부순환로,...,60,NaN,NaN,99.0,2008.0,서울지방경찰청,02-700-5031,2020-05-31,1320000,경찰청


In [50]:
data3 = data3[["무인교통단속카메라관리번호","위도","경도"]]

In [51]:
data3

,무인교통단속카메라관리번호,위도,경도
0,F9956,37.474269,126.991028
1,F9957,37.457198,127.013242
2,F9342,37.480058,126.993807
3,F9352,37.485751,127.044071
4,F9360,37.511017,127.007223
...,...,...,...
91,F4729,37.488571,127.082170
92,F4736,37.524000,127.047768
93,F5035,37.487367,127.062321
94,F4416,37.488197,127.100312


In [52]:
data3['sub_cnt']=1
data3['경도'] = data3['경도'].astype(float)
data3['위도'] = data3['위도'].astype(float)
data3['point'] = data3.apply(lambda row : Point([row['경도'], row['위도']]), axis=1)
data3 = gpd.GeoDataFrame(data3, geometry='point')

<ipython-input-52-9f078b38371f>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data3['sub_cnt']=1
<ipython-input-52-9f078b38371f>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data3['경도'] = data3['경도'].astype(float)
<ipython-input-52-9f078b38371f>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html

In [53]:
data3

,무인교통단속카메라관리번호,위도,경도,sub_cnt,point
0,F9956,37.474269,126.991028,1,POINT (126.99103 37.47427)
1,F9957,37.457198,127.013242,1,POINT (127.01324 37.45720)
2,F9342,37.480058,126.993807,1,POINT (126.99381 37.48006)
3,F9352,37.485751,127.044071,1,POINT (127.04407 37.48575)
4,F9360,37.511017,127.007223,1,POINT (127.00722 37.51102)
...,...,...,...,...,...
91,F4729,37.488571,127.082170,1,POINT (127.08217 37.48857)
92,F4736,37.524000,127.047768,1,POINT (127.04777 37.52400)
93,F5035,37.487367,127.062321,1,POINT (127.06232 37.48737)
94,F4416,37.488197,127.100312,1,POINT (127.10031 37.48820)


In [54]:
gid_list = []
for i in tqdm(range(data3.shape[0])):
    point = data3['point'].iloc[i]
    if data_gid['geometry'].contains(point).sum() == 0:
        g1 = "Na"
    else :
        g = data_gid['gid'].loc[data_gid['geometry'].contains(point)]
        g1 = g.iloc[0]
    gid_list.append(g1)

100%|██████████████████████████████████████████████████████████████████████████████████| 96/96 [00:14<00:00,  6.41it/s]


In [55]:
data3['gid'] = gid_list
data3['TL_cnt']=1
result3 = data3[['TL_cnt', 'gid']].groupby(['gid']).sum()
result3.reset_index(inplace=True)
data3 =result3


In [56]:
data3

,gid,TL_cnt
0,다사542430,1
1,다사542436,1
2,다사543443,1
3,다사546413,1
4,다사546431,1
...,...,...
89,다사647407,1
90,다사651418,1
91,다사652406,1
92,다사653419,1


In [57]:
data3.to_csv("output/그리드 전처리 완료/서초강남무인교통단속카메라전처리완료.csv")